In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import PIL
#Allow memory growth 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

tf.keras.backend.set_learning_phase(0)

def create_model(): 
    IMG_SHAPE = (40, 40, 3) #kHWC
    print("Load mobile net")
    # Create the base model from the pre-trained model MobileNet V2
    base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, 
                                                   input_tensor=tf.keras.layers.Input(shape=(40, 40, 3), batch_size=1),
                                                   include_top=False, weights='imagenet')
    #for layer in Infer_model.layers:
    #    print( layer._name )
    #    if(str("mobilenetv2_1.00_160") in layer._name):
    #        print("changed name!")
    #        layer._name = str("mobilenetv2_1_00_160")
    print("Append last layers")
    model = tf.keras.Sequential([
      base_model,
      keras.layers.GlobalAveragePooling2D(),
      keras.layers.Dense(43),
      keras.layers.Activation('softmax')
    ])
    print("compile model")
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    print("done")
    return model
                                                   
Infer_model = create_model()

input_placeholder = Infer_model.get_layer(index = 0).get_layer(index = 0)
print("Input Dimensions")
print(input_placeholder.batch_size)
print(input_placeholder.input_shape)
print(input_placeholder.name)

print("Load weights")
Infer_model.load_weights("./model/weights_v1_40x40_v01_tf")

print("Model architecture")
Infer_model.summary() 

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import graph_io
        
print("\n Freeze Graph \n")

def freeze_graph(graph, session, output):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, ".", "frozen_model_v1_40x40.pb", as_text=False)


session = tf.keras.backend.get_session()

INPUT_NODE = Infer_model.inputs[0].op.name
OUTPUT_NODE = Infer_model.outputs[0].op.name

print(INPUT_NODE)
print(OUTPUT_NODE)

freeze_graph(session.graph, session, [out.op.name for out in Infer_model.outputs])